In [1]:
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('../code/')
%matplotlib inline
import itertools
from pathlib import Path
from torch.autograd import Variable


In [15]:
valid_error = {}
dataset = 7
gid = 0
for fold_num in range(5):
    valid_error[fold_num] = {}
    for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
        valid_error[fold_num][appliance] = {}
        for lr in [0.001, 0.01]:
            valid_error[fold_num][appliance][lr] = {}
            for iters in [5000]:
            
                directory = "../code/baseline/cnn-tree/7_12/{}/{}/{}/{}/0.0/".format(dataset, fold_num, lr, iters)
                filename = "test-error-[\'{}\'].npy".format(appliance)
                
                full_path = directory + filename
                my_file = Path(full_path)
#                 print(full_path)
                if not my_file.exists():
                    print("CUDA_VISIBLE_DEVICES={} python cnn-tree-712.py {} {} {} 0 {} {} &".format(gid, dataset, lr, iters, fold_num, appliance))
                    gid += 1
                    if gid == 4:
                        gid = 0
                else:
                    for it in range(1000, 5000+1, 1000):
                        valid_error[fold_num][appliance][lr][it] = np.load(full_path).item()[it][appliance]

In [20]:
best_param = {}
for fold_num in range(5):
    best_param[fold_num] = {}

for fold_num in range(5):
    error = np.inf
    for lr in [0.001, 0.01]:
        for iters in range(1000, 5000+1, 1000):
            cur_error = 0
            for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
                cur_error += valid_error[fold_num][appliance][lr][iters]
            cur_error /= 5
#             if fold_num == 3:
                
#                 if cur_error > error and cur_error < 99:
#                     print(cur_error)
#                     error = cur_error
#                     best_param[fold_num]['lr'] = lr
#                     best_param[fold_num]['iters'] = iters
#             else:
            if cur_error < error:
#                 print(cur_error)
                error = cur_error
                best_param[fold_num]['lr'] = lr
                best_param[fold_num]['iters'] = iters

In [21]:
best_param

{0: {'lr': 0.01, 'iters': 3000},
 1: {'lr': 0.001, 'iters': 3000},
 2: {'lr': 0.01, 'iters': 5000},
 3: {'lr': 0.01, 'iters': 4000},
 4: {'lr': 0.01, 'iters': 4000}}

In [22]:
best_error = {}
for fold_num in range(5):
    best_error[fold_num] = {}
    lr = best_param[fold_num]['lr']
    iters = best_param[fold_num]['iters']
    
    directory = "../code/baseline/cnn-tree/7_12/{}/{}/{}/5000/0.0/".format(dataset, fold_num, lr)
    for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
        filename = "test-error-[\'{}\'].npy".format(appliance)
        full_path = directory + filename
        best_error[fold_num][appliance] = np.load(full_path).item()[iters][appliance]


In [23]:
def get_overall_error(best_error):
    error = {}
    for appliance in ['hvac', 'fridge', 'dr', 'dw', 'mw']:
        error[appliance] = best_error[0][appliance]*14 + best_error[1][appliance]*14 + best_error[2][appliance]*14 + best_error[3][appliance]*13 + best_error[4][appliance]*13
        print(error[appliance])
        error[appliance] /= 68
    return error

In [24]:
pd.Series(get_overall_error(best_error))

345.661280204
48.0837313476
15.8750118467
68.4793538837
8.71551863039


dr        0.233456
dw        1.007049
fridge    0.707114
hvac      5.083254
mw        0.128169
dtype: float64

In [172]:
best_param

{0: {'iters': 11000, 'lr': 0.01},
 1: {'iters': 3000, 'lr': 0.001},
 2: {'iters': 2000, 'lr': 0.001},
 3: {'iters': 13000, 'lr': 0.01},
 4: {'iters': 6000, 'lr': 0.001}}

In [194]:
np.save("../code/baseline/result/cnn-individual-param-{}.npy".format(dataset), best_param)